In [9]:
import sys, os
import pandas as pd
import numpy as np
sys.path.append(str(_dh[0]) +'/Source')
from SafeBoundUtils import *
from JoinGraphUtils import *

np.random.seed(100) #########

def compute_true_cardinality(edge_tables):

    print(f"{list(edge_tables.keys())}")
    visited_edges = set()
    edge_names = list(edge_tables.keys())

    result = edge_tables[edge_names[0]].copy()
    visited_edges.add(edge_names[0])
    edge_names.remove(edge_names[0])
    print(f"add{edge_names[0]}")

    while edge_names:
        for e_name in edge_names:
            if e_name in visited_edges:
                continue

            current_table = edge_tables[e_name]

            for column in current_table.columns:
                if column in result.columns:
                    # 检测自环,单独处理
                    columns_list = current_table.columns.tolist()
                    parts = columns_list[1].split('_')
                    if len(parts) >= 2:
                        column_name = parts[0]
                        suffix = parts[-1]
                        if suffix.startswith("dup"):
                            result = result.merge(current_table, on = column_name)
                            result.drop(columns=[f"{column_name}_dup"], inplace=True)
                            tmp = pd.DataFrame()
                            tmp[column_name] = current_table[f"{column_name}_dup"]
                            tmp[f"{column_name}_dup"] = current_table[column_name]
                            result = result.merge(tmp, on = column_name)
                            result.drop(columns=[f"{column_name}_dup"], inplace=True)
                            visited_edges.add(e_name)
                            edge_names.remove(e_name)
                            break

                    result = result.merge(current_table)
                    print(f"add{e_name}")
                    visited_edges.add(e_name)
                    edge_names.remove(e_name)
                    break

    return len(result)

In [10]:
R = pd.DataFrame()
R["X"] = np.random.randint(1, 100, 1000)
S = pd.DataFrame()
S["X"] = np.random.randint(1, 100, 3000)
S["Y"] = np.random.randint(1, 1000, 3000)
T = pd.DataFrame()
T["Y"] = np.array(range(0,1000))
T["A"] =  np.random.randint(1, 100, 1000)
M = pd.DataFrame()
M["A"] = np.random.randint(1, 100, 3000)
M["O"] = np.random.randint(1, 1000, 3000)
N = pd.DataFrame()
N["O"] = np.random.randint(1, 100, 3000)
N["X"] = np.random.randint(1, 1000, 3000)
A = pd.DataFrame()
A["X"] = np.random.randint(1, 100, 3000)
A["X_dup"] = np.random.randint(1, 1000, 3000)

tableDFs = [R, S, T,A]
tableNames = ["R", "S", "T","A"]
tableJoinCols = [["X"], ["X", "Y"], ["Y"],["X"]]
# filterColumns = [[], [], ["A"]]
FKtoKDict = {"S": [["Y", "T", "Y"]],"R":[["X","S","X"]]}  # S has a FK-PK relationship with T on the attribute Y

edge_table = {}
edge_table["R"] = R
edge_table["S"] = S
edge_table["T"] = T
edge_table["M"] = M
edge_table["N"] = N
edge_table["A"] = A

trueCardinality = len(R.merge(S, on="X").merge(T, on="Y").merge(M, on="A").merge(N).merge(A,on="X").merge(A,left_on="X",right_on="X_dup",suffixes=("","_dup_dup")))

# Building the statistics object
safeBound = SafeBound(tableDFs=tableDFs, tableNames=tableNames, tableJoinCols=tableJoinCols, relativeErrorPerSegment=.01, FKtoKDict=FKtoKDict, numCores=2)

In [11]:
# Defining a query
query = JoinQueryGraph()
# query.addJoin("R","X","S", "X")
# query.addJoin("S","Y","T", "Y")
query.addAlias("R","R1")
query.addAlias("R","R2")
query.addAlias("S","S1")
query.addAlias("S","S2")
query.addJoin("R1","X","S1", "X")
query.addJoin("R2","X","S2", "X")
query.addJoin("R2","X","S1", "X")
query.addJoin("S1","Y","T", "Y")
query.addJoin("A","X","R1", "X")
#query.addPredicate("T","A","<", 20)
query.buildJoinGraph()
query.printJoinGraph()

R1 is the 1th joined table with predicates: 
R1 joins with R2 on column X
R1 joins with S1 on column X
R1 joins with A on column X
A is the 2th joined table with predicates: 
A joins with S1 on column X
A joins with R1 on column X
S1 is the 3th joined table with predicates: 
S1 joins with R2 on column X
S1 joins with S2 on column X
S1 joins with A on column X
S1 joins with T on column Y
S1 joins with R1 on column X
T is the 4th joined table with predicates: 
T joins with S1 on column Y
S2 is the 5th joined table with predicates: 
S2 joins with R2 on column X
S2 joins with S1 on column X
R2 is the 6th joined table with predicates: 
R2 joins with S2 on column X
R2 joins with R1 on column X
R2 joins with S1 on column X


In [12]:
for i in range(1,10):
    bound = safeBound.functionalFrequencyBound(query)
    # print("True Cardinality: " + str(trueCardinality))
    # print(f"true card:{compute_true_cardinality(edge_table)}")
    print("Cardinality Bound: " + str(bound))
print("SafeBound Memory (kB): " + str(safeBound.memory()/1000))

Cardinality Bound: 553835265.8772068
Cardinality Bound: 553835265.8772068
Cardinality Bound: 553835265.8772068
Cardinality Bound: 553835265.8772068
Cardinality Bound: 553835265.8772068
Cardinality Bound: 553835265.8772068
Cardinality Bound: 553835265.8772068
Cardinality Bound: 553835265.8772068
Cardinality Bound: 553835265.8772068
Table: R
Table: S
Table: T
Table: A
SafeBound Memory (kB): 0.84
